## 25.4:投資と退職の問題
あなたは65歳であり、１００万ドルを保有している。あなたは投資の組み合わせを決めようとしている。年率6%の無リスク再建と、期待される対数リターンが8%、リターンの標準偏差が30%のリスクのある株式ポートフォリオがある。制約条件は、毎年15万ドルを口座から引き出し、75歳の時点でいくらかの資産を残したいということである。

In [105]:
import numpy as np
import pandas as pd

In [155]:
#初期条件設定
a0 = 1000000
nonrisk_return = 0.06
risk_return = 0.08
risk_var = 0.3
risk_ratio = 0.7
annual_withdrawal = 150000

In [181]:
#年初の財産
assets_begginy = np.zeros(10)
assets_begginy[0] = a0
#リスク資産への投資
investment_to_risk = np.zeros(10)
investment_to_risk[0] = assets_begginy[0] * risk_ratio
#債券への投資
investment_to_nonrisk = np.zeros(10)
investment_to_nonrisk[0] = assets_begginy[0] * (1-risk_ratio)
#1+リスク資産からのリターン
return_from_risk = np.zeros(10)
return_from_risk[0] = np.exp(risk_return + Z[0] * risk_var)
#年末の資産
assets_endy = np.zeros(10)
assets_endy[0] = investment_to_risk[0]*return_from_risk[0] + investment_to_nonrisk[0]*np.exp(nonrisk_return)

#10年後の財産
final_asset = 0

In [182]:
#数値計算
for i in range(1, 10):
    assets_begginy[i] = assets_endy[i-1] - annual_withdrawal
    investment_to_risk[i] = assets_begginy[i] * risk_ratio
    investment_to_nonrisk[i] = assets_begginy[i] * (1-risk_ratio)
    return_from_risk[i] =risk_return + (np.exp(risk_return + Z[i] * risk_var))
    assets_endy[i] = investment_to_risk[i] * return_from_risk[i] + investment_to_nonrisk[i] * np.exp(non_risk_return)
final_asset = assets_endy[9]-annual_withdrawal

In [183]:
#DataFrameに当てはめる
retirement_plan = pd.DataFrame({"年初の財産":assets_begginy, 
                                "リスク資産への投資":investment_to_risk,
                                "債券への投資":investment_to_nonrisk,
                                "正規乱数":Z,
                                "リスク資産のリターン":return_from_risk,
                                "年末の財産":assets_endy,
                                "引き出し":annual_withdrawal},
                              index=[1,2,3,4,5,6,7,8,9,10])

In [184]:
#負の値を赤で表示させる
def highlight_negative(val):
    if val < 0:
        return 'color: {0}; font-weight: bold'.format('red')
    else:
        return 'color: {0}'.format('black')


In [185]:
print("１０年後の財産:",final_asset)
retirement_plan.style.applymap(highlight_negative)


１０年後の財産: -748257.9951195953


,年初の財産,リスク資産への投資,債券への投資,正規乱数,リスク資産のリターン,年末の財産,引き出し
1,1e+06,700000,300000,-1.06878,0.786129,868841,150000
2,718841,503189,215652,-0.630453,0.976609,720406,150000
3,570406,399284,171122,-1.78333,0.714449,466971,150000
4,316971,221880,95091.4,0.154474,1.21467,370483,150000
5,220483,154338,66144.8,-0.53444,1.00281,225006,150000
6,75006.4,52504.5,22501.9,-0.02738,1.15443,84505.9,150000
7,-65494.1,-45845.9,-19648.2,-0.0959855,1.13254,-72785.4,150000
8,-222785,-155950,-66835.6,0.232941,1.2417,-264611,150000
9,-414611,-290228,-124383,-0.164374,1.11116,-454565,150000
10,-604565,-423196,-181370,-0.698104,0.958595,-598258,150000


#### 乱数の出方によっては10年後に資産が残らなくなってしまう 
70%ルールをやみくもに使うのは良くないのかもしれない

毎年の引き出し額に制約をつけてみる（年末の財産が150,000*3以上なら150,000を引き出し、未満なら全財産の1/3を引き出す）

In [186]:
#安全弁を設定
safety_valve = 3
withdrawals = []

In [150]:
Z = np.random.randn(10)

In [187]:
for i in range(1, 10):
    if assets_endy[i-1]>annual_withdrawal*safety_valve:
        assets_begginy[i] = assets_endy[i-1] - annual_withdrawal
        withdrawals.append(annual_withdrawal)
    else:
        assets_begginy[i] = assets_endy[i-1]*2/3
        withdrawals.append(assets_endy[i-1]/3)
        
    investment_to_risk[i] = assets_begginy[i] * risk_ratio
    investment_to_nonrisk[i] = assets_begginy[i] * (1-risk_ratio)
    return_from_risk[i] =risk_return + ((Z[i] * risk_var) + 1)
    assets_endy[i] = investment_to_risk[i] * return_from_risk[i] + investment_to_nonrisk[i] * np.exp(non_risk_return)
    


In [188]:
if assets_endy[9]>annual_withdrawal*safety_valve:
    final_asset = assets_endy[9]-annual_withdrawal
    withdrawals.append(annual_withdrawal)
else:
    final_asset = assets_endy[9]*2/3
    withdrawals.append(annual_withdrawal/3)

In [189]:
#DataFrameに当てはめる
retirement_plan = pd.DataFrame({"年初の財産":assets_begginy, 
                                "リスク資産への投資":investment_to_risk,
                                "債券への投資":investment_to_nonrisk,
                                "正規乱数":Z,
                                "1+リスク資産のリターン":return_from_risk,
                                "年末の財産":assets_endy,
                                "引き出し":withdrawals},
                              index=[1,2,3,4,5,6,7,8,9,10])

In [190]:
print("１０年後の財産:",final_asset)
retirement_plan.style.applymap(highlight_negative)


１０年後の財産: 18746.480905560067


,年初の財産,リスク資産への投資,債券への投資,正規乱数,1+リスク資産のリターン,年末の財産,引き出し
1,1e+06,700000,300000,-1.06878,0.786129,868841,150000
2,718841,503189,215652,-0.630453,0.890864,677260,150000
3,527260,369082,158178,-1.78333,0.545001,369110,123037
4,246073,172251,73821.9,0.154474,1.12634,272401,90800.2
5,181600,127120,54480.1,-0.53444,0.919668,174757,58252.5
6,116505,81553.4,34951.5,-0.02738,1.07179,124521,41506.9
7,83013.7,58109.6,24904.1,-0.0959855,1.0512,87529.2,29176.4
8,58352.8,40846.9,17505.8,0.232941,1.14988,65557.5,21852.5
9,43705,30593.5,13111.5,-0.164374,1.03069,45454.6,15151.5
10,30303.1,21212.2,9090.93,-0.698104,0.870569,28119.7,50000
